In [ ]:
#@author Shray Alag
#@version 08.05.2020
#@mentor Abhi
#@runtime running the whole notebook takes less than 10 seconds, but remember to input your choice of which motifs it
#should get the co-occurance of
#Finds the co-occurances of two motifs and runs statistics using the kmer sequences in a fasta file. Displays all
#of the meme motif sequences (in the sequence logo format) and the meme motifs information (e-value, etc.). Asks user
#which of thsoe two motifs they would like to get co-occurance of and then reads through the fasta file and uses a dict
#to store the information about the length difference between when the two motifs occur. Then, it creates histograms.
#The doExtraAnalysis feature allows analysis of negative and positive lengths (when the second motif occurs before the
#first).



#You need to install biopython (pip install biopython) and logomaker (pip install logomaker)


import matplotlib.pyplot as plt
from pandas import DataFrame
import logomaker as logo
from Bio.Seq import Seq
from Bio import SeqIO
import pandas as pd
import numpy as np
%matplotlib inline
import time
import re
import os




#TO USE CHANGE BASE_DIR and the other file directories.
fastaFile = "ZNF134_ChipSeq.fa"



#T.ATCAG.TGA.GGTGG
#GGTGG.........T.ATCAG.TGA
#GGTGG.........T.AT.AG.TGA
#T.AT.AG..GA
motifList = []
infoMotifList = []
firstMotifIndex = -2
firstMotifStr= input("Please Enter the Motif's signature. Use a period to indicate where the TF is not very specific. For example, T.ATCAG.TGA.GGTGG could be an inputed motif. The motif you inputted is: ")
# secondMotifStr =""
secondMotifStr= input("Please Enter the Motif's signature. Use a period to indicate where the TF is not very specific. For example, T.ATCAG.TGA.GGTGG could be an inputed motif. The motif you inputted is: ")
bothOccuranceLength = {}
doExtraAnalysis = True
global_time = time.time()

In [ ]:
#takes in a 2D Pandas Dataframe
def getSpecialEncoding(motifs):
    rows = []
    for index, row in motifs.iterrows():
        rows.append((computationNeg(row["A"]), computationNeg(row["C"]), computationNeg(row["G"]), computationNeg(row["T"])))
    df = pd.DataFrame(rows, columns=['A', 'C', 'G', 'T'])
    return df
        
    
    
    
    
    
def computationNeg(coloumn):
    temp = float(coloumn)
    if temp < 0.15:
        return (temp - 0.10)/1.75
    if temp > 0.8:
        return ((temp)*4)/1.75
    if temp > 0.6:
        return ((temp)*3.5)/1.75
    if temp > 0.35:
        return ((temp*3))/1.75
    else:
        return (temp -0.1)/1.75

In [ ]:

def readInFasta(firstMotifStr, secondMotifStr, bothOccuranceLength):
    twelveSequenceDict = {}
    negThirteenSequence = {}
    negThirteenSequenceFull = {}
    posThirteenSequence = {}
    bothOccuranceLength = {}
    counter = 0
    relevantCounter = 0
    print(len(firstMotifStr))
    print(firstMotifStr)
    print(len(secondMotifStr))
    print(secondMotifStr)
    
    
    fasta_sequences = SeqIO.parse(open(fastaFile),'fasta')
    
    for fasta in fasta_sequences:
        counter = counter + 1
        name, sequence = fasta.id, str(fasta.seq)
        sequence = sequence.upper()
        firstFound = re.search(firstMotifStr, sequence)
        secondFound = re.search(secondMotifStr, sequence)
        
        
#         if counter < 100:
#             print(sequence)
            
        if not firstFound or not secondFound:
            sequence = str(Seq(sequence).reverse_complement())
            firstFound = re.search(firstMotifStr, sequence)
        
        
        #secondFound.start()+len(secondFound)
        while firstFound and secondFound:
            
            length = firstFound.start() - secondFound.start()
            relevantCounter = relevantCounter + 1
            
            if length < 0:
                print(length, sequence[firstFound.start():(secondFound.start() + len(secondMotifStr))])
            else:
                print(length, sequence[secondFound.start():(firstFound.start() + len(firstMotifStr))])
                
            if length in bothOccuranceLength:
                bothOccuranceLength.update({length: (bothOccuranceLength.get(length) + 1)})       
            else:
                bothOccuranceLength.update({length: 1})
                
            if sequence[firstFound.start():firstFound.start()+len(firstMotifStr)] in negThirteenSequence:
                negThirteenSequence.update({sequence[firstFound.start():firstFound.start()+len(firstMotifStr)]: (negThirteenSequence.get(sequence[firstFound.start():firstFound.start()+len(firstMotifStr)]) + 1)})       
            else:
                negThirteenSequence.update({sequence[firstFound.start():firstFound.start()+len(firstMotifStr)]: 1})
            
            if sequence[secondFound.start():secondFound.start()+len(secondMotifStr)] in posThirteenSequence and firstFound.start() != 0:
                posThirteenSequence.update({sequence[secondFound.start():secondFound.start()+len(secondMotifStr)] : (posThirteenSequence.get(sequence[secondFound.start():secondFound.start()+len(secondMotifStr)]) + 1)})
            else:
                posThirteenSequence.update({sequence[secondFound.start():secondFound.start()+len(secondMotifStr)] : 1})
            
            sequence = sequence[firstFound.start()+len(firstMotifStr):]
            firstFound = re.search(firstMotifStr, sequence)
            secondFound = re.search(secondMotifStr, sequence)
    bothOccuranceLength = {k: v for k, v in sorted(bothOccuranceLength.items(), key=lambda item: item[1])}
    posThirteenSequence = {k: v for k, v in sorted(posThirteenSequence.items(), key=lambda item: item[1])}
#     print(bothOccuranceLength)
    print(counter)
    print(bothOccuranceLength)
    print("num of occurrences: ", relevantCounter)

    return bothOccuranceLength, posThirteenSequence




    
bothOccuranceLength, posThirteenSequence = readInFasta(firstMotifStr, secondMotifStr, bothOccuranceLength)

In [ ]:
posLength = {}
negLength = {}



for key in bothOccuranceLength:
    if int(key) > 0:
        posLength.update({key: bothOccuranceLength.get(key)})
    else:
        negLength.update({key: bothOccuranceLength.get(key)})
            
            
plt.bar(bothOccuranceLength.keys(), bothOccuranceLength.values(), 3, color='g')

In [ ]:
plt.bar(posLength.keys(), posLength.values(), 1, color='b')

In [ ]:
plt.bar(negLength.keys(), negLength.values(), 1, color='r')

In [ ]:
import random
from sklearn.preprocessing import normalize

def convert_to_one_hot(sequence, count):
    convert_dict = {'a':(float(count),0.0,0.0,0.0),
                   'c':(0.0,float(count),0.0,0.0),
                   'g':(0.0,0.0,float(count),0.0),
                   't':(0.0,0.0,0.0,float(count)),
                   'n':(0.25,0.25,0.25,0.25)}
    sequence = sequence.lower()
    arr = [convert_dict[s] for s in sequence]
    arr = np.vstack(arr)
    return(arr)

def convert_one_hot_to_seq(one_hot_arr):
    convert_dict = {(1,0,0,0):'A',(0,1,0,0):'C',(0,0,1,0):'G',(0,0,0,1):'T'}
    seq = ''
    for i in one_hot_arr:
        seq+=convert_dict[tuple(i)]
    return(seq)

def concatenate(negProbabilities, key, count):
    return np.concatenate((negProbabilities, convert_to_one_hot(key, count)), axis=0)

# negProbabilitiesList = []
# count = 0
# for key in negThirteenSequence:
#     if count == 0:
#         negProbabilitiesList = convert_to_one_hot(key, negThirteenSequence.get(key))
#     else: 
#         negProbabilitiesList = negProbabilitiesList + convert_to_one_hot(key, negThirteenSequence.get(key))
#     count = count + 1


# negProbabilitiesList = normalize(negProbabilitiesList, axis=1, norm='l1')

# logo_logo = logo.Logo(getSpecialEncoding(pd.DataFrame(np.asarray(negProbabilitiesList),
#                 columns=['A', 'C', 'G', 'T'])), color_scheme={'A': 'red','C': 'blue', 'G': 'orange', 'T': 'green'},baseline_width=0,
#                            show_spines=False,
#                            vsep=.005,
#                            width=.80)

# logo_logo = logo.Logo(pd.DataFrame(np.asarray(negProbabilitiesList),
#                 columns=['A', 'C', 'G', 'T']), color_scheme={'A': 'red','C': 'blue', 'G': 'orange', 'T': 'green'},baseline_width=0,
#                            show_spines=False,
#                            vsep=.005,
#                            width=.80)

In [ ]:
rows, cols = (1000, 4) 
thirteenList = [[0]*cols]*rows

count = 0
for key in bothOccuranceLength:
    one_hot_result = convert_to_one_hot(key, bothOccuranceLength.get(key))
    if len(one_hot_result) != 1000:
        print(len(one_hot_result))
        rows, cols = (1000 - len(one_hot_result), 4)
        extra = np.asarray([[0]*cols]*rows)
        one_hot_result = np.vstack((extra, one_hot_result))  
    thirteenList = thirteenList + one_hot_result
    count = count + 1


thirteenList = normalize(thirteenList, axis=1, norm='l1')
thirteenList = thirteenList[~np.all(thirteenList == 0, axis=1)]

logo_logo = logo.Logo(getSpecialEncoding(pd.DataFrame(np.asarray(thirteenList),
                columns=['A', 'C', 'G', 'T'])), color_scheme={'A': 'red','C': 'blue', 'G': 'orange', 'T': 'green'},baseline_width=0,
                           show_spines=False,
                           vsep=.005,
                           width=.80)


thirteenList = np.flip(thirteenList)
np.save("/home/shray/ZNF134_numpy.npy", thirteenList)
logo_logo = logo.Logo(getSpecialEncoding(pd.DataFrame(np.asarray(thirteenList),
                columns=['A', 'C', 'G', 'T'])), color_scheme={'A': 'red','C': 'blue', 'G': 'orange', 'T': 'green'},baseline_width=0,
                           show_spines=False,
                           vsep=.005,
                           width=.80)

In [ ]:
print(thirteenList)